## これはなに？
個人的な学習のため、[こちら](https://www.kaggle.com/jagangupta/time-series-basics-exploring-traditional-ts)を翻訳しながら内容をつけ加えていったものです。

(日本語に直しながら読むの面倒！という方々向けに役立てば望外の喜びです。voteはリンク先の方に差し上げてください)

## 時系列解析の基礎:

従来の様々な時系列モデルの基本的な概念と、その背景にある基本的な考え方とを照らし合わせながら、理解を深めていきます。

## 目的:
「このカーネルは、初心者のための時系列に関する様々な概念の保存場所として作られましたが、専門家の方にも確認用として役立つことを願っています :)」

とのことです(冒頭リンク先の著者の方より)。ありがたい限りですね。。

## もくじ:
* コンペとデータの概要
* データとパッケージのインポート
* 基本的な探査/EDA
* 単一の時系列
    * 定常性
    * 季節性、トレンド、残余
    * AR , MA , ARMA , ARIMA
    * AICを用いたPとQの選択
    * ETS
    * Prophet
    * UCM
* 階層型時系列
    * ボトムアップ
    * AHP
    * PHA 
    * FP 
    
    
## コンペとデータの概要:

このコンペでは、ロシアのソフトウェア会社の来月の全製品・全店舗の総売上高を予測するという課題が与えられています。-[1c company](http://1c.ru/eng/title.htm). 

**対象となるソフトウェア会社は何をやるか?:**

1C: Enterprise 8システムのプログラムは、日常的な企業活動の自動化を目的としています。管理会計、ビジネス会計、人事管理、CRM、SRM、MRPなど、経済・経営活動のさまざまなビジネスタスクを対象としています。

**データ**:
店舗と商品の組み合わせごとに日次の売上データが用意されていますが、今回の課題は月次レベルでの売上予測です。

## インポート:


In [ ]:
# まず、ファイルをチェックすることから始めましょう
!ls ../input/*

In [ ]:
# 基本的なパッケージ
import numpy as np # 線形代数
import pandas as pd # データの加工, CSVファイルのI/O (e.g. pd.read_csv)
import random as rd # 乱数の生成
import datetime # manipulating date formats
# 可視化用
import matplotlib.pyplot as plt # basic plotting
import seaborn as sns # for prettier plots


# 時系列解析用
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs


# 設定
import warnings
warnings.filterwarnings("ignore")



In [ ]:
# 全てインポート
sales=pd.read_csv("../input/sales_train.csv")

# 設定
import warnings
warnings.filterwarnings("ignore")

item_cat=pd.read_csv("../input/item_categories.csv")
item=pd.read_csv("../input/items.csv")
sub=pd.read_csv("../input/sample_submission.csv")
shops=pd.read_csv("../input/shops.csv")
test=pd.read_csv("../input/test.csv")

In [ ]:
# 日付カラムを正しくフォーマットする
sales.date=sales.date.apply(lambda x:datetime.datetime.strptime(x, '%d.%m.%Y'))
# 確認
print(sales.info())

In [ ]:
# 必要なメトリクスを月次レベルで集計

monthly_sales=sales.groupby(["date_block_num","shop_id","item_id"])[
    "date","item_price","item_cnt_day"].agg({"date":["min",'max'],"item_price":"mean","item_cnt_day":"sum"})

## コードをブレイクダウンして理解していきます:
# 日付ブロック（月）とshop_id、item_idによる集計
# date,item_price,item_cnt(sales)の各カラムを選択
# どのカラムに対してどのような集約を行うかを示す辞書を用意
# 日付の最小値と最大値
# item_priceの平均値
# 売上高の合計

In [ ]:
# take a peak
monthly_sales.head(20)

In [ ]:
# カテゴリごとのアイテム数
x=item.groupby(['item_category_id']).count()
x=x.sort_values(by='item_id',ascending=False)
x=x.iloc[0:10].reset_index()
x
# plot
plt.figure(figsize=(8,4))
ax= sns.barplot(x.item_category_id, x.item_id, alpha=0.8)
plt.title("Items per Category")
plt.ylabel('# of items', fontsize=12)
plt.xlabel('Category', fontsize=12)
plt.show()

もちろん、このデータセットには他にもたくさんの発見がありますが、時系列のパートに進んでみましょう。

# 単一の時系列:

目的は、ある店舗とアイテムの組み合わせにおける翌月の売上を予測することです。

各店舗アイテムの経時的な売上は、それ自体が時系列です。すべての組み合わせを調べる前に、まず単一の時系列を予測する方法を理解しましょう。

ここでは、会社全体の月ごとの総売上高を予測することにします。

まず、月ごとの総売上高を計算し、そのデータをプロットしてみましょう。


In [ ]:
ts=sales.groupby(["date_block_num"])["item_cnt_day"].sum()
ts.astype('float')
plt.figure(figsize=(16,8))
plt.title('Total Sales of the company')
plt.xlabel('Time')
plt.ylabel('Sales')
plt.plot(ts);

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(ts.rolling(window=12,center=False).mean(),label='Rolling Mean');
plt.plot(ts.rolling(window=12,center=False).std(),label='Rolling sd');
plt.legend();

**パッと見てわかること:**
明らかに「季節性」（例：ある時期に売上がピークになる）があり、「トレンド」は減少しています。

そこで、トレンド、季節性、残差に分解して確認してみましょう。


In [ ]:
import statsmodels.api as sm
# 積算
res = sm.tsa.seasonal_decompose(ts.values,freq=12,model="multiplicative")
#plt.figure(figsize=(16,12))
fig = res.plot()
#fig.show()

In [ ]:
# 加算モデル
res = sm.tsa.seasonal_decompose(ts.values,freq=12,model="additive")
#plt.figure(figsize=(16,12))
fig = res.plot()
#fig.show()

In [ ]:
# Rバージョンのpythonへの移植

# alas ! rpy2 does not exist in Kaggle kernals :( 
# from rpy2.robjects import r
# def decompose(series, frequency, s_window, **kwargs):
#     df = pd.DataFrame()
#     df['date'] = series.index
#     s = [x for x in series.values]
#     length = len(series)
#     s = r.ts(s, frequency=frequency)
#     decomposed = [x for x in r.stl(s, s_window, **kwargs).rx2('time.series')]
#     df['observed'] = series.values
#     df['trend'] = decomposed[length:2*length]
#     df['seasonal'] = decomposed[0:length]
#     df['residual'] = decomposed[2*length:3*length]
#     return df

加法モデルを仮定すると、次のように書けます。

> yt=St+Tt+Et 

ここでytは期間tのデータ、Stは期間tの季節成分、Ttは期間ttのトレンド・サイクル成分、Etは期間tの残余（またはイレギュラー、エラー）成分です。
Multiplicativeモデルも同様です。

> yt=St  x Tt x Et 

## 定常性:

![q](https://static1.squarespace.com/static/53ac905ee4b003339a856a1d/t/5818f84aebbd1ac01c275bac/1478031479192/?format=750w)

Stationarity（定常性）とは、時系列の時間的不変性を意味します。すなわち、時系列の2つの点は、互いにどれだけ離れているかだけで関係しており、方向（前進／後退）によっては関係していません。

時系列が定常の場合、モデル化が容易になります。統計的なモデリング手法は、時系列が定常であることを前提としたり、必要としたりします。

定常性をチェックするためには、複数の検定があります。
* ADF( Augmented Dicky Fuller Test) 
* KPSS 
* PP (Phillips-Perron test)

ここでは、最も一般的に使用されているADFを実行してみましょう。

注: [ExcelでDicky Fullerテストを行うためのガイド](http://www.real-statistics.com/time-series-analysis/stochastic-processes/dickey-fuller-test/)

[その他の便利なガイド](http://www.blackarbs.com/blog/time-series-analysis-in-python-linear-models-to-garch/11/1/2016#AR) 

[オススメのリファレンス](https://github.com/ultimatist/ODSC17/blob/master/Time%20Series%20with%20Python%20(ODSC)%20STA.ipynb)


In [ ]:
# 定常性の検定
def test_stationarity(timeseries):
    
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

test_stationarity(ts)


In [ ]:
# トレンドを取り除く
from pandas import Series as Series
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)

# invert differenced forecast
def inverse_difference(last_ob, value):
    return value + last_ob



In [ ]:
ts=sales.groupby(["date_block_num"])["item_cnt_day"].sum()
ts.astype('float')
plt.figure(figsize=(16,16))
plt.subplot(311)
plt.title('Original')
plt.xlabel('Time')
plt.ylabel('Sales')
plt.plot(ts)
plt.subplot(312)
plt.title('After De-trend')
plt.xlabel('Time')
plt.ylabel('Sales')
new_ts=difference(ts)
plt.plot(new_ts)
plt.plot()

plt.subplot(313)
plt.title('After De-seasonalization')
plt.xlabel('Time')
plt.ylabel('Sales')
new_ts=difference(ts,12)       # 季節性は12か月とおく
plt.plot(new_ts)
plt.plot()

In [ ]:
# 季節性を除いた上で、改めて定常性の検定を行う
test_stationarity(new_ts)

### 変換後、DF検定のp値は5%以内に収まっています。したがって、系列の定常性を仮定することができます。

先ほど定義した逆変換関数を使えば、簡単に元のシリーズに戻すことができます。

それでは早速、予測を行ってみましょう。

# AR, MA, ARMA:
TL: モデルのDR版:

MA - 系列の次の値は、前のn個の値の平均の関数
AR - 次の値の誤差(平均値の差)は、前のn個の値の誤差の関数
ARMA - 両者の混合

では、時系列がARプロセスなのかMAプロセスなのかを調べるにはどうすればよいのでしょうか。

早速見てみましょう！

In [ ]:
def tsplot(y, lags=None, figsize=(10, 8), style='bmh',title=''):
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    with plt.style.context(style):    
        fig = plt.figure(figsize=figsize)
        #mpl.rcParams['font.family'] = 'Ubuntu Mono'
        layout = (3, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))
        qq_ax = plt.subplot2grid(layout, (2, 0))
        pp_ax = plt.subplot2grid(layout, (2, 1))
        
        y.plot(ax=ts_ax)
        ts_ax.set_title(title)
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.5)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.5)
        sm.qqplot(y, line='s', ax=qq_ax)
        qq_ax.set_title('QQ Plot')        
        scs.probplot(y, sparams=(y.mean(), y.std()), plot=pp_ax)

        plt.tight_layout()
    return 

In [ ]:
# AR(1)過程をalpha = 0.6でシミュレーション
np.random.seed(1)
n_samples = int(1000)
a = 0.6
x = w = np.random.normal(size=n_samples)

for t in range(n_samples):
    x[t] = a*x[t-1] + w[t]
limit=12    
_ = tsplot(x, lags=limit,title="AR(1)process")

## AR(1) process -- has ACF tailing out and PACF cutting off at lag=1

In [ ]:
# AR(2)過程をシミュレーションする

n = int(1000)
alphas = np.array([.444, .333])
betas = np.array([0.])

# Pythonでは、ゼロラグの値を1に指定する必要があります。
# ARモデルのアルファは否定されなければならないことにも注意してください。
# AR(p)モデルの場合は、MAのベータ値を0に設定します。
# 詳細は statsmodels.org の例を参照してください。

ar = np.r_[1, -alphas]
ma = np.r_[1, betas]

ar2 = smt.arma_generate_sample(ar=ar, ma=ma, nsample=n) 
_ = tsplot(ar2, lags=12,title="AR(2) process")

## AR(2) process -- has ACF tailing out and PACF cutting off at lag=2

In [ ]:
# MA(1)過程をシミュレーションする
n = int(1000)
# set the AR(p) alphas equal to 0
alphas = np.array([0.])
betas = np.array([0.8])
# add zero-lag and negate alphas
ar = np.r_[1, -alphas]
ma = np.r_[1, betas]
ma1 = smt.arma_generate_sample(ar=ar, ma=ma, nsample=n) 
limit=12
_ = tsplot(ma1, lags=limit,title="MA(1) process")

## MA(1) process -- has ACF cut off at lag=1

In [ ]:
# MA(2)過程をbetas=0.6, 0.4でシミュレーションする
n = int(1000)
alphas = np.array([0.])
betas = np.array([0.6, 0.4])
ar = np.r_[1, -alphas]
ma = np.r_[1, betas]

ma3 = smt.arma_generate_sample(ar=ar, ma=ma, nsample=n)
_ = tsplot(ma3, lags=12,title="MA(2) process")

## MA(2) process -- has ACF cut off at lag=2

In [ ]:
# ARMA(2, 2)モデルをalphas=[0.5,-0.25], betas=[0.5,-0.3]でシミュレーションする
max_lag = 12

n = int(5000) # lots of samples to help estimates
burn = int(n/10) # number of samples to discard before fit

alphas = np.array([0.8, -0.65])
betas = np.array([0.5, -0.7])
ar = np.r_[1, -alphas]
ma = np.r_[1, betas]

arma22 = smt.arma_generate_sample(ar=ar, ma=ma, nsample=n, burnin=burn)
_ = tsplot(arma22, lags=max_lag,title="ARMA(2,2) process")

## ここから先は少しぼんやりしています。あまり明確ではありません。

上記のプロットをまとめると、以下のようになります:

ACF Shape	| Indicated Model |
-- | -- |
指数関数的に、ゼロまで減衰する |	自己回帰モデル。部分自己相関プロットを使って自己回帰モデルの次数を特定する |
正と負を交互に繰り返す、ゼロに減衰する 自己回帰モデル |  部分自己相関プロットを使用して順序を特定する |
1つ以上のスパイク、残りは基本的にゼロ | 移動平均モデル、プロットがゼロになる場所で順序を識別 |
数回のラグの後に減衰 |	混合自己回帰・移動平均（ARMA）モデル | 
すべてゼロまたはゼロに近い | データは基本的にランダム |
一定の間隔で高い値が出る | 季節性自己回帰項を含む |
ゼロまで減衰しない |	系列は定常的でない |


## ARとMAのプロセスの順番を見つけるために、システム的なアプローチをしてみましょう。

In [ ]:
# aicで最適な順番を選ぶ 
# 最小のaic値が勝つ
best_aic = np.inf 
best_order = None
best_mdl = None

rng = range(5)
for i in rng:
    for j in rng:
        try:
            tmp_mdl = smt.ARMA(arma22, order=(i, j)).fit(method='mle', trend='nc')
            tmp_aic = tmp_mdl.aic
            if tmp_aic < best_aic:
                best_aic = tmp_aic
                best_order = (i, j)
                best_mdl = tmp_mdl
        except: continue


print('aic: {:6.5f} | order: {}'.format(best_aic, best_order))


## シミュレーションされたプロセスの次数がARMA(2,2)であることを正しく認識しました。

### これをセールスの時系列データに使ってみましょう。


In [ ]:
# aicで最適な順番を選ぶ 
# 最小のaic値が勝つ
best_aic = np.inf 
best_order = None
best_mdl = None

rng = range(5)
for i in rng:
    for j in rng:
        try:
            tmp_mdl = smt.ARMA(new_ts.values, order=(i, j)).fit(method='mle', trend='nc')
            tmp_aic = tmp_mdl.aic
            if tmp_aic < best_aic:
                best_aic = tmp_aic
                best_order = (i, j)
                best_mdl = tmp_mdl
        except: continue


print('aic: {:6.5f} | order: {}'.format(best_aic, best_order))


In [ ]:
# best_mdl.predict()を使って次の値を予測するだけです。

In [ ]:
# 日付をインデックスとして時系列に追加する
ts=sales.groupby(["date_block_num"])["item_cnt_day"].sum()
ts.index=pd.date_range(start = '2013-01-01',end='2015-10-01', freq = 'MS')
ts=ts.reset_index()
ts.head()

# Prophet: 

最近、Facebookの研究によってオープンソース化されました。これは非常に有望なツールで、イライラする**フラットライン**の解決に、しばしば非常に便利で迅速な解決策となります。

![フラットライン](https://i.stack.imgur.com/fWzyX.jpg)

確かに、適切な前処理を行い、パラメータを慎重に調整すれば、上のグラフのようなことは起こらないと言うこともできます。

しかし、実際のところ、私たちのほとんどは、それを実現するための忍耐力も専門知識も持ち合わせていません。

また、ほとんどの実用的なシナリオでは、予測が必要な時系列がたくさんあるという事実があります。
例えば、今回のコンペでは、店舗とアイテムレベルの組み合わせについて、翌月の売上を予測する必要がありますが、その数は数千にも及ぶ可能性があります。

もう1つの優れた機能は、典型的な**sklearn**の構文に従っていることです。

Prophetメソッドの核となるのは、4つの主要なコンポーネントからなる加法回帰モデルです。

* ピースワイズ・リニアまたはロジスティック成長曲線のトレンド。Prophetは、データからチェンジポイントを選択することで、トレンドの変化を自動的に検出します。
* フーリエ級数を使ってモデル化された年間の季節成分
* ダミー変数を使った週単位の季節成分
* ユーザーが提供した重要な祝日のリスト

**prophetについて詳しく知るためのリソース:**
* https://www.youtube.com/watch?v=95-HMzxsghY
* https://facebook.github.io/prophet/docs/quick_start.html#python-api
* https://research.fb.com/prophet-forecasting-at-scale/
* https://blog.exploratory.io/is-prophet-better-than-arima-for-forecasting-time-series-fa9ae08a5851

In [ ]:
from fbprophet import Prophet
# prophetは以下の設定でpandas dfを必要とします。
# ( date column named as DS and the value column as Y)
ts.columns=['ds','y']
model = Prophet( yearly_seasonality=True) #instantiate Prophet with only yearly seasonality as our data is monthly 
model.fit(ts) #fit the model with your dataframe

In [ ]:
# predict for five months in the furure and MS - month start is the frequency
future = model.make_future_dataframe(periods = 5, freq = 'MS')  
# now lets make the forecasts
forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
model.plot(forecast)

In [ ]:
model.plot_components(forecast)

凄いですね。プロフェットからのトレンドや季節感は、以前に伝統的な方法で行ったものと似ています。

## UCM:

Unobserved Components Modelの略です。ここでの直感は、prophetのそれと似ています。このモデルでは、時系列をトレンド、季節、サイクルといった構成要素に分解し、それらを回帰させ、構成要素の次のポイントを予測し、それらを組み合わせます。

残念ながら、Pythonでこのモデルを実行できる良いパッケージ/コードは見つかりませんでした :( 

UCMのRバージョン: https://bicorner.com/2015/12/28/unobserved-component-models-in-r/

# 階層型時系列:

The [Forecasting: principles and practice](https://www.otexts.org/fpp/9/4) , は、Rob J Hyndman氏による予測のための究極の参考書です。
彼は、グループ化された、あるいは階層的な予測を扱うための基本を説明しています。次のような簡単なシナリオを考えてみましょう。

Hyndmanはこの階層のポイントを推定するために以下の方法を提案しています。直感的に理解できるように言葉を簡略化してみました。

### ボトムアップアプローチ:
* 任意の方法でベースレベルのシリーズをすべて予測し、それをトップに集約するだけです。
* 利点: 単純であること、集計によって情報が失われないこと。
* 不利な点: 下位レベルはノイズが多い

### トップダウンアプローチ:
* トップレベルを最初に予測する。(例：総売上高を最初に予測する)
* 次に、ベースレベルの予測に与える必要のある総売上高の割合を示す**重み**を計算します（例：総売上高に対するアイテムの売上の貢献度）。
* 「ウェイト」を算出する方法はさまざまです。
    * **過去の平均比率** - 過去数ヶ月間のアイテムの売上貢献度の単純平均値
    * **過去の平均値に占める割合** - Weightは、下位シリーズの平均値と全シリーズの平均値の比率です。
    * **予想される割合** - 過去の割合の変化から将来の割合を予測する
* これらのウェイトを使って、ベースとなる予測値やその他のレベルを算出します。

### ミドルアウト:
* ボトムアップとトップダウンの両方を併用する。
* 例：店舗-アイテムレベルの予測を行う問題を考えてみましょう。
    * 中間レベル（店舗）を取り、店舗の予測を見つける。
    * ボトムズアップで全体の売上高を求める。
    * 比例を使って店舗の売上を分解し、トップダウン・アプローチでアイテムレベルの売上を求める。
    
### 最適な組み合わせのアプローチ:
* すべての層を独立して予測する
* すべての層が独立しているため、階層間の整合性が取れない可能性があります。
    * 例：アイテムが独立して予測されているため、店舗で販売されたアイテムの合計は、店舗の予測販売額とは一致しない可能性がありますが、ヒンドマンは「集約的な一貫性」と表現しています。
* 予測を階層と整合させるためのアドホックな調整を行うために、いくつかのマトリックス計算と調整が行われます。


### 理論はもういい。予測を始めましょう！
今回の問題では、アイテム数が22170、ストア数が60です。つまり、予測すべき時系列（アイテムとストアの組み合わせ）は、**約100万個**もあるということです。

それらを一つ一つ設定するのはほとんど不可能です。そこで、それを代行してくれるProphetを使いましょう。

ボトムズアップのアプローチで始めましょう。

ここでは他にも考慮すべき点があります。
* すべての店舗がすべての商品を販売しているわけではない
* 新商品が出たらどうする？
* 製品が棚から取り除かれた場合は？

In [ ]:
total_sales=sales.groupby(['date_block_num'])["item_cnt_day"].sum()
dates=pd.date_range(start = '2013-01-01',end='2015-10-01', freq = 'MS')

total_sales.index=dates
total_sales.head()

In [ ]:
# 月次の販売データからアイテム・ストアのユニークな組み合わせを取得する
monthly_sales=sales.groupby(["shop_id","item_id","date_block_num"])["item_cnt_day"].sum()
# arrange it conviniently to perform the hts 
monthly_sales=monthly_sales.unstack(level=-1).fillna(0)
monthly_sales=monthly_sales.T
dates=pd.date_range(start = '2013-01-01',end='2015-10-01', freq = 'MS')
monthly_sales.index=dates
monthly_sales=monthly_sales.reset_index()
monthly_sales.head()

In [ ]:
import time
start_time=time.time()

# Bottoms up
# Calculating the base forecasts using prophet
# From HTSprophet pachage -- https://github.com/CollinRooney12/htsprophet/blob/master/htsprophet/hts.py
forecastsDict = {}
for node in range(len(monthly_sales)):
    # take the date-column and the col to be forecasted
    nodeToForecast = pd.concat([monthly_sales.iloc[:,0], monthly_sales.iloc[:, node+1]], axis = 1)
#     print(nodeToForecast.head())  # just to check
# rename for prophet compatability
    nodeToForecast = nodeToForecast.rename(columns = {nodeToForecast.columns[0] : 'ds'})
    nodeToForecast = nodeToForecast.rename(columns = {nodeToForecast.columns[1] : 'y'})
    growth = 'linear'
    m = Prophet(growth, yearly_seasonality=True)
    m.fit(nodeToForecast)
    future = m.make_future_dataframe(periods = 1, freq = 'MS')
    forecastsDict[node] = m.predict(future)
    if (node== 10):
        end_time=time.time()
        print("forecasting for ",node,"th node and took",end_time-start_time,"s")
        break
    

~10回の予測で～16秒。100万回の予測が必要です。これではうまくいきません。

# ミドルアウト:
店舗レベルで予測してみましょう

In [ ]:
monthly_shop_sales=sales.groupby(["date_block_num","shop_id"])["item_cnt_day"].sum()
# get the shops to the columns
monthly_shop_sales=monthly_shop_sales.unstack(level=1)
monthly_shop_sales=monthly_shop_sales.fillna(0)
monthly_shop_sales.index=dates
monthly_shop_sales=monthly_shop_sales.reset_index()
monthly_shop_sales.head()

In [ ]:
start_time=time.time()

# Calculating the base forecasts using prophet
# From HTSprophet pachage -- https://github.com/CollinRooney12/htsprophet/blob/master/htsprophet/hts.py
forecastsDict = {}
for node in range(len(monthly_shop_sales)):
    # take the date-column and the col to be forecasted
    nodeToForecast = pd.concat([monthly_shop_sales.iloc[:,0], monthly_shop_sales.iloc[:, node+1]], axis = 1)
#     print(nodeToForecast.head())  # just to check
# rename for prophet compatability
    nodeToForecast = nodeToForecast.rename(columns = {nodeToForecast.columns[0] : 'ds'})
    nodeToForecast = nodeToForecast.rename(columns = {nodeToForecast.columns[1] : 'y'})
    growth = 'linear'
    m = Prophet(growth, yearly_seasonality=True)
    m.fit(nodeToForecast)
    future = m.make_future_dataframe(periods = 1, freq = 'MS')
    forecastsDict[node] = m.predict(future)
    

In [ ]:
#predictions = np.zeros([len(forecastsDict[0].yhat),1]) 
nCols = len(list(forecastsDict.keys()))+1
for key in range(0, nCols-1):
    f1 = np.array(forecastsDict[key].yhat)
    f2 = f1[:, np.newaxis]
    if key==0:
        predictions=f2.copy()
       # print(predictions.shape)
    else:
       predictions = np.concatenate((predictions, f2), axis = 1)

In [ ]:
predictions_unknown=predictions[-1]
predictions_unknown

## 脚注:

私は統計学を専攻しているわけではないので、重要な技術が抜けていたり、内容に間違いがあったりしたら、コメントで教えてください。

時系列に関する別のカーネルをここに追加する予定で、最近の時系列コンペティション（Favorita、Recruitなど）のオープンソースソリューションをこのプレイグラウンドデータセットに適応させることを考えています。

コメントやアップヴォートをお願いします。) 